In [2]:
import os
import numpy    as np
import pandas   as pd
import psycopg2 as psy

from datetime import datetime as dt
from random   import randrange
from random   import sample
from random   import choice

/usr/local/lib/python3.7/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [3]:
#''' Check if file exists on path. Move existing file to ARCH. Create new input for consultant_dim.'''


In [4]:
# Setup script input parameters

min_obs = 100                      # minimal number of observation in output dataset
max_obs = 110                     # maximal number of observation in output dataset

obs = randrange(min_obs,max_obs) # number of observation
obs

109

In [5]:
main_path    = "Documents/CallCenterStaffing"
path_to_arch = "Arch/consultant"
path_to_file = "Input"
path_to_dict = "Dict"
file_name    = "consultant_dim.csv"
name_dict    = "name_dict.csv"

now          = dt.now()

path_to_file = os.path.join(os.path.expanduser("~"),main_path,path_to_file,file_name)
path_to_arch = os.path.join(os.path.expanduser("~"),main_path,path_to_arch)
name_dict    = os.path.join(os.path.expanduser("~"),main_path,path_to_dict,name_dict)

def is_path_not_correct(path):
    return not(os.path.exists(path))

def is_path_correct(path):
    return os.path.exists(path)

if is_path_not_correct(path_to_file):
    raise Exception("Input file doesn't exist")

if is_path_not_correct(path_to_arch):
    raise Exception("Archive directory doesn't exist")
        
if is_path_not_correct(name_dict):
    raise Exception("Name dictionary doesn't exist")

time_stamp = "_" + str(now)[:10]
arch_file_name = 'arch_' + file_name.replace(".csv",time_stamp + ".csv")
 
os.rename(path_to_file, os.path.join(path_to_arch, arch_file_name))

In [6]:
# connection to database and return list of column column_list
try:
    connection = psy.connect( user = "szymonbocian",
                              password = "",
                              host = "localhost",
                              port = "5432",
                              database = "dwh_call_center")

    cursor = connection.cursor()

    query = "SELECT column_name FROM INFORMATION_SCHEMA.columns WHERE table_schema = 'stg' AND table_name = 'consultant_dim';"
    cursor.execute(query)
    column_list = cursor.fetchall()
    
    query = "SELECT DISTINCT consultant_key FROM pro.consultant_dim WHERE manager_key IS NULL;"
    cursor.execute(query)
    manager_list = cursor.fetchall()
    
except (Exception, psy.Error) as error :
    print ("Error while connecting to PostgreSQL", error)
finally:
        if(connection):
            cursor.close()
            connection.close()

column_list = [''.join(c for c in s if c not in '(),') for s in column_list]

a = list()
for row in manager_list:
    a.append(row[0])

manager_list = a
    
print(column_list)
print(manager_list)

['consultant_first_name', 'consultant_second_name', 'consultant_last_name', 'manager_key', 'consultant_business_key']
[4]


In [7]:
# Load dictionary with first_name (second_name), last_name
dict_name = pd.read_csv( name_dict,
                         sep = ";",
                         usecols = ['male_name','female_name','last_name'],
                         skiprows = 0
                       )

In [12]:
f = pd.concat([dict_name.male_name,dict_name.female_name])

df = pd.DataFrame({
    "first_name"  : sample(tuple(f), k = obs),
    "is_sec_name" : np.random.choice([True, False], obs, True, [0.15, 0.85])
})

df['is_male']   = df.first_name.isin(dict_name.male_name)
df['is_female'] = df.first_name.isin(dict_name.female_name)

df['second_name'] = df.apply(lambda l: choice(dict_name.male_name) if l['is_male'] and l['is_sec_name'] else "", axis = 1)
df['second_name'] = df.apply(lambda l: choice(dict_name.female_name) if l['is_female'] and l['is_sec_name'] else l['second_name'], axis = 1)

df['last_name'] = sample(tuple(dict_name.last_name),k = obs)

df['manager_key'] = np.random.choice(manager_list)

In [13]:
# Create consultant business key 
fnl = ('00' + (df.first_name.str.len() *8).astype(str))
lnl = ('00' + (df.last_name.str.len() *8).astype(str))
cal = ('00' + df.manager_key.astype(str))

df['consultant_business_key'] = df.first_name.map(str).str.slice(0,1) + df.last_name.map(str).str.slice(0,1) + [w[-3:] for w in fnl] + [w[-3:] for w in lnl] + [w[-3:] for w in cal] + "E"

In [14]:
df = df.drop(['is_sec_name', 'is_male', 'is_female'], axis = 1)
df

,first_name,second_name,last_name,manager_key,consultant_business_key
0,Wincent,,Partyka,4,WP056056004E
1,Milan,,Moskal,4,MM040048004E
2,Polikarp,,Michalak,4,PM064064004E
3,Eliza,,Gondz,4,EG040040004E
4,Miłosława,,Gut,4,MG072024004E
...,...,...,...,...,...
104,Jacek,,Unrug,4,JU040040004E
105,Zachariasz,,Paluszkiewicz,4,ZP080104004E
106,Helen,,Smolar,4,HS040048004E
107,Mateusz,Bartłomiej,Rusek,4,MR056040004E


In [15]:
# Create csv file with given destination

df.columns = column_list
df.to_csv(path_to_file, index = None, header = True)